In [ ]:
import pandas as pd
import numpy as np
import psutil
import matplotlib.pyplot as plt
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from pathlib import Path
import math
import os, sys

notebook_dir = Path().cwd()
project_root = notebook_dir.parent
sys.path.insert(0, str(project_root))
import importlib
import user_function as uf

In [ ]:
path0 = '/mnt/sda1/RA5/data'
path1 = '/mnt/sda1/RA5/intermediate/siyoung/Event_study'

Random Data

In [ ]:
ds_random_shc = pd.read_parquet(os.path.join(path0, 'random_hshd_Shc_all_1.parquet'))
# Running time: 10m
ds_random_shc = uf.clean_data(ds_random_shc)

In [2]:
print(ds_random_shc.shape)
shc_hshd_list = ds_random_shc[ds_random_shc['ta_ym'].notna()].HSHD_SEQNO.unique()
ds_random_shc = ds_random_shc[ds_random_shc['HSHD_SEQNO'].isin(shc_hshd_list)]
print(ds_random_shc.shape)

# (227987560, 67)
# (187491323, 67)

In [ ]:
ds_random_shc['is_f_20_40'] = (ds_random_shc['SEX'] == 'Female') & (ds_random_shc['AGE'].between(20,40))
ds_random_shc['is_m_20_40'] = (ds_random_shc['SEX'] == 'Male') & (ds_random_shc['AGE'].between(20,40))

ds_random_shc['hshd_is_two_parents'] = ((ds_random_shc.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_f_20_40'].transform('max') == 1) &
                                       (ds_random_shc.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_m_20_40'].transform('max') == 1))
ds_random_shc['n_obs'] = ds_random_shc.groupby(['HSHD_SEQNO','BS_YR_MON'])['KEY'].transform('count')

ds_random_shc = ds_random_shc[(ds_random_shc['hshd_is_two_parents'] == True) &
                              (ds_random_shc['n_obs'] == 2)]
ds_random_shc.sort_values(['HSHD_SEQNO','BS_YR_MON','KEY'], inplace = True)


# birth 제외하기
ds_random_shc['hshd_has_no_birth'] = (ds_random_shc.groupby('HSHD_SEQNO')['shc_ch_birth'].transform('max') == 0)
ds_random_shc = ds_random_shc[ds_random_shc['hshd_has_no_birth']]

In [ ]:
print(ds_random_shc.shape)


# print(ds_random_shc.shape)

In [ ]:
np.save(os.path.join(path1, 'shc_hshd_list.npy'), shc_hshd_list)
ds_random_shc.to_parquet(os.path.join(path1,'ds_random_shc.parquet'),index=False, compression='snappy')

In [ ]:
ds_random_kcb = pd.read_parquet(os.path.join(path0, 'random_hshd_KCB_all_1.parquet'))
ds_random_kcb = uf.kcb_preprocess(ds_random_kcb)
ds_random_kcb = uf.clean_data(ds_random_kcb)

In [ ]:
shc_hshd_list = np.load(
                            os.path.join(path1, "shc_hshd_list.npy"),
                            allow_pickle=True
                        )
print(ds_random_kcb.shape)
ds_random_kcb = ds_random_kcb[ds_random_kcb.HSHD_SEQNO.isin(shc_hshd_list)]
print(ds_random_kcb.shape)


# (227987560, 65)
# (187491323, 65)

In [ ]:
ds_random_kcb['is_f_20_40'] = (ds_random_kcb['SEX'] == 'Female') & (ds_random_kcb['AGE'].between(20,40))
ds_random_kcb['is_m_20_40'] = (ds_random_kcb['SEX'] == 'Male') & (ds_random_kcb['AGE'].between(20,40))

ds_random_kcb['hshd_is_two_parents'] = ((ds_random_kcb.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_f_20_40'].transform('max') == 1) &
                                       (ds_random_kcb.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_m_20_40'].transform('max') == 1))
ds_random_kcb['n_obs'] = ds_random_kcb.groupby(['HSHD_SEQNO','BS_YR_MON'])['KEY'].transform('count')

ds_random_kcb = ds_random_kcb[(ds_random_kcb['hshd_is_two_parents'] == True) &
                              (ds_random_kcb['n_obs'] == 2)]

print(ds_random_kcb.shape)

# (15462184, 69)

In [ ]:
ds_random_hshd = ds_random_kcb.groupby(['HSHD_SEQNO', 'BS_YR_MON']).agg({'ICM':'sum', 'TOT_ASST':'sum'}).reset_index()
ds_random_hshd['ICM'] = ds_random_hshd['ICM'].replace(0,np.nan)
ds_random_hshd = ds_random_hshd.groupby('HSHD_SEQNO').agg({'ICM':'mean', 'TOT_ASST':'mean'}).reset_index()
ds_random_hshd['ICM'] = ds_random_hshd['ICM'].replace(0,np.nan)
_, income_bins = pd.qcut(ds_random_hshd['ICM'], q=5, retbins=True, duplicates='drop')
_, asset_bins = pd.qcut(ds_random_hshd['TOT_ASST'], q=5, retbins=True, duplicates='drop')

kcb_cutoffs = pd.DataFrame({
    **{f'ICM_cutoff{i}': [v] for i, v in enumerate(income_bins)},
    **{f'TOT_ASST_cutoff{i}': [v] for i, v in enumerate(asset_bins)}})

In [ ]:
kcb_cutoffs.to_excel(os.path.join(path1,'kcb_cutoffs.xlsx'))
ds_random_kcb.to_parquet(os.path.join(path1,'ds_random_kcb.parquet'))

In [ ]:
ds_random_shc = pd.read_parquet(os.path.join(path1,'ds_random_shc.parquet'))
ds_random_kcb = pd.read_parquet(os.path.join(path1,'ds_random_kcb.parquet'))
print(ds_random_kcb.shape)
print(ds_random_shc.shape)

# (15462184, 69)
# (13976956, 72)


ds_random_combined = ds_random_kcb.merge(ds_random_shc, on=['KEY','HSHD_SEQNO','BS_YR_MON'], how='inner')
print(ds_random_combined.shape)

# (13976956, 138)

In [ ]:
home_mode = (
    ds_random_combined
    .groupby("HSHD_SEQNO")["HOME_ADM"]
    .agg(lambda x: x.mode().iloc[0])
    .reset_index()
)
home_mode['HOME_ADM'] = home_mode["HOME_ADM"].astype(str)
home_mode['HOME_ADM_2'] = home_mode['HOME_ADM'].str[:2]
home_mode['HOME_ADM_5'] = home_mode['HOME_ADM'].str[:5]
home_mode.rename(columns={'HOME_ADM':'HOME_ADM_b'}, inplace=True)

ds_random_combined = ds_random_combined.merge(home_mode, on=['HSHD_SEQNO'], how='left')
ds_random_combined.to_parquet(os.path.join(path1,'ds_random_combined.parquet'),index=False, compression='snappy')

In [ ]:
ds_random_skt = (pd.read_parquet(os.path.join(path0,'random_hshd_skt_all_2.parquet'),
                columns=['KEY','BS_YR_MON','SEX','AGE','HSHD_SEQNO','GENDER','COMM_SNS_DAY','INFO_PORTAL_DAY']))


In [ ]:
shc_hshd_list = np.load(
                            os.path.join(path1, "shc_hshd_list.npy"),
                            allow_pickle=True
                        )
print(ds_random_skt.shape)
ds_random_skt = uf.skt_preprocess(ds_random_skt)
ds_random_skt = ds_random_skt[ds_random_skt.HSHD_SEQNO.isin(shc_hshd_list)]
ds_random_skt = uf.clean_data(ds_random_skt)
print(ds_random_skt.shape)

# (227987560, 8)
# (187491323, 9)

In [ ]:
ds_random_skt['is_f_20_40'] = (ds_random_skt['SEX'] == 'Female') & (ds_random_skt['AGE'].between(20,40))
ds_random_skt['is_m_20_40'] = (ds_random_skt['SEX'] == 'Male') & (ds_random_skt['AGE'].between(20,40))

ds_random_skt['hshd_is_two_parents'] = ((ds_random_skt.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_f_20_40'].transform('max') == 1) &
                                       (ds_random_skt.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_m_20_40'].transform('max') == 1))
ds_random_skt['n_obs'] = ds_random_skt.groupby(['HSHD_SEQNO','BS_YR_MON'])['KEY'].transform('count')

ds_random_skt = ds_random_skt[(ds_random_skt['hshd_is_two_parents'] == True) &
                              (ds_random_skt['n_obs'] == 2)]

print(ds_random_skt.shape)

# (15462184, 13)

In [ ]:
ds_random_indiv = ds_random_skt.groupby(['KEY']).agg({'COMM_SNS_DAY':'mean', 'INFO_PORTAL_DAY':'mean'}).reset_index()

ds_random_combined = pd.read_parquet(os.path.join(path1,'ds_random_combined.parquet'))
ds_random_combined = ds_random_combined.merge(ds_random_indiv, on = 'KEY', how='left')

ds_random_combined.to_parquet(os.path.join(path1,'ds_random_combined.parquet'),index=False, compression='snappy')

In [ ]:
# Cutoff value 구하기

_, sns_bins = pd.qcut(ds_random_indiv['COMM_SNS_DAY'], q=2, retbins=True, duplicates='drop')
_, portal_bins = pd.qcut(ds_random_indiv['INFO_PORTAL_DAY'], q=2, retbins=True, duplicates='drop')

skt_cutoffs = pd.DataFrame({
    **{f'sns_cutoff{i}': [v] for i, v in enumerate(sns_bins)},
    **{f'portal_cutoff{i}': [v] for i, v in enumerate(portal_bins)}
})
skt_cutoffs.to_excel(os.path.join(path1,'skt_cutoffs.xlsx'))

In [ ]:
# Birth Household
ds_birth_shc = pd.read_parquet(os.path.join(path0,'shc_birth_Shc_all_1.parquet'))
ds_birth_shc['BS_YR_MON'] = pd.to_datetime(ds_birth_shc['BS_YR_MON'].str.strip(), format='%Y%m')
print(ds_birth_shc.shape)
key_sex_pair = pd.read_parquet(os.path.join(path0, 'shc_birth_KCB_all.parquet'))[['KEY','BS_YR_MON','AGE','SEX']].drop_duplicates()
key_sex_pair = uf.clean_data(key_sex_pair)
ds_birth_shc = ds_birth_shc.merge(key_sex_pair, on=['KEY','BS_YR_MON'], how='inner')
print(ds_birth_shc.shape)

birth_hshd_list = ds_birth_shc[ds_birth_shc['ta_ym'].notna()].HSHD_SEQNO.unique()
print(ds_birth_shc.shape)
ds_birth_shc = ds_birth_shc[ds_birth_shc.HSHD_SEQNO.isin(birth_hshd_list)]
print(ds_birth_shc.shape)
np.save(os.path.join(path1, 'shc_birth_hshd_list.npy'), birth_hshd_list)

In [ ]:
df_hshd_birth_months = pd.read_parquet(os.path.join(path1,'hshd_birth_month.parquet')).drop(columns=['birth_order'])
ds_birth_shc = ds_birth_shc.merge(df_hshd_birth_months, on=['HSHD_SEQNO'], how='left')
ds_birth_shc['BS_YR_MON_birth'].isna().sum()

In [ ]:
ds_birth_shc['is_f_20_40'] = (ds_birth_shc['SEX'] == 'Female') & (ds_birth_shc['AGE'].between(20,40))
ds_birth_shc['is_m_20_40'] = (ds_birth_shc['SEX'] == 'Male') & (ds_birth_shc['AGE'].between(20,40))

ds_birth_shc['hshd_is_two_parents'] = ((ds_birth_shc.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_f_20_40'].transform('max') == 1) &
                                       (ds_birth_shc.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_m_20_40'].transform('max') == 1))
ds_birth_shc['n_obs'] = ds_birth_shc.groupby(['HSHD_SEQNO','BS_YR_MON'])['KEY'].transform('count')

ds_birth_shc = ds_birth_shc[(ds_birth_shc['hshd_is_two_parents'] == True) &
                              (ds_birth_shc['n_obs'] == 2)]
ds_birth_shc.sort_values(['HSHD_SEQNO','BS_YR_MON','KEY'], inplace = True)
print(ds_birth_shc.shape)

# (5099834, 72)

In [ ]:
ds_birth_kcb = pd.read_parquet(os.path.join(path0, 'shc_birth_KCB_all.parquet'))
ds_birth_kcb = uf.kcb_preprocess(ds_birth_kcb)
ds_birth_kcb = uf.clean_data(ds_birth_kcb)
print(ds_birth_kcb.shape)
ds_birth_kcb = ds_birth_kcb.loc[ds_birth_kcb.HSHD_SEQNO.isin(birth_hshd_list)]
print(ds_birth_kcb.shape)

In [ ]:
ds_birth_kcb['is_f_20_40'] = (ds_birth_kcb['SEX'] == 'Female') & (ds_birth_kcb['AGE'].between(20,40))
ds_birth_kcb['is_m_20_40'] = (ds_birth_kcb['SEX'] == 'Male') & (ds_birth_kcb['AGE'].between(20,40))

ds_birth_kcb['hshd_is_two_parents'] = ((ds_birth_kcb.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_f_20_40'].transform('max') == 1) &
                                       (ds_birth_kcb.groupby(['HSHD_SEQNO','BS_YR_MON'])['is_m_20_40'].transform('max') == 1))
ds_birth_kcb['n_obs'] = ds_birth_kcb.groupby(['HSHD_SEQNO','BS_YR_MON'])['KEY'].transform('count')

ds_birth_kcb = ds_birth_kcb[(ds_birth_kcb['hshd_is_two_parents'] == True) &
                              (ds_birth_kcb['n_obs'] == 2)]
print(ds_birth_kcb.shape)

ds_birth_combined = ds_birth_kcb.merge(ds_birth_shc, on=['KEY','HSHD_SEQNO','BS_YR_MON']
                                       ,how='inner')
print(ds_birth_combined.shape)

# (5099834, 139)
# (5099834, 208)

ds_birth_combined.to_parquet(os.path.join(path1,'ds_birth_combined.parquet'),index=False, compression='snappy')

In [ ]:
ds_birth_combined['months_from_birth'] = ((ds_birth_combined['BS_YR_MON'].dt.year - ds_birth_combined['BS_YR_MON_birth'].dt.year) * 12 +
                                   (ds_birth_combined['BS_YR_MON'].dt.month - ds_birth_combined['BS_YR_MON_birth'].dt.month))

ds_birth_combined['max_months_from_birth_before'] = (
    ds_birth_combined
    .assign(tmp=lambda d: d['months_from_birth'].where(d['months_from_birth'] <= 0))
    .groupby('HSHD_SEQNO')['tmp']
    .transform('max')
)

hshd_birth_adm = (ds_birth_combined[ds_birth_combined['months_from_birth']
                    == ds_birth_combined['max_months_from_birth_before']]
                    [['HSHD_SEQNO','HOME_ADM']].drop_duplicates())

hshd_birth_adm['HOME_ADM'] = hshd_birth_adm['HOME_ADM'].astype(str)
hshd_birth_adm['HOME_ADM_2'] = hshd_birth_adm['HOME_ADM'].str[:2]
hshd_birth_adm['HOME_ADM_5'] = hshd_birth_adm['HOME_ADM'].str[:5]
hshd_birth_adm.rename(columns={'HOME_ADM':'HOME_ADM_b'},inplace=True)

ds_birth_combined = ds_birth_combined.merge(hshd_birth_adm, on='HSHD_SEQNO', how='left')
print(ds_birth_combined.shape)

ds_birth_combined.to_parquet(os.path.join(path1,'ds_birth_combined.parquet'),index=False, compression='snappy')



In [ ]:
ds_birth_skt = (pd.read_parquet(os.path.join(path0,'shc_birth_skt_all.parquet'),
                columns=['KEY','BS_YR_MON','SEX','AGE','HSHD_SEQNO','GENDER','COMM_SNS_DAY','INFO_PORTAL_DAY']))

print(ds_birth_skt.shape)
ds_birth_skt = uf.skt_preprocess(ds_birth_skt)
ds_birth_skt = uf.clean_data(ds_birth_skt)
print(ds_birth_skt.shape)

In [ ]:
ds_birth_indiv = ds_birth_skt.groupby('KEY').agg({'COMM_SNS_DAY':'mean', 'INFO_PORTAL_DAY':'mean'})
ds_birth_combined = ds_birth_combined.merge(ds_birth_indiv, on='KEY', how = 'left')
ds_birth_combined.to_parquet(os.path.join(path1,'ds_birth_combined.parquet'))

In [ ]:
ds_random_combined = pd.read_parquet(os.path.join(path1,'ds_random_combined.parquet'))
ds_birth_combined = pd.read_parquet(os.path.join(path1,'ds_birth_combined.parquet'))
ds_birth_combined['ICM'] = ds_birth_combined['ICM'].replace(0, np.nan)

ds_birth_combined['flag_birth'] = 1
ds_random_combined['flag_birth'] = 0
ds_combined = pd.concat([ds_birth_combined,ds_random_combined], axis=0)
ds_combined['birth event'] = (ds_combined['BS_YR_MON'] == ds_combined['BS_YR_MON_birth']).astype('int')
 # 이상치 제거
ds_combined = ds_combined[~ds_combined['HOME_ADM_b'].isna()]
ds_combined.to_parquet(os.path.join(path1,'ds_combined.parquet'))

Add heterogeneity

In [ ]:
# Household ICM & TOT_ASST
ds_combined = pd.read_parquet(os.path.join(path1,'ds_combined.parquet'))

ds_hshd = ds_combined.groupby(['HSHD_SEQNO','BS_YR_MON']).agg({'ICM':'sum', 'TOT_ASST':'sum'}).reset_index()
ds_hshd['ICM'] = ds_hshd['ICM'].replace(0, np.nan)
ds_hshd = ds_hshd.groupby('HSHD_SEQNO').agg({'ICM':'mean', 'TOT_ASST':'mean'}).reset_index()
ds_hshd['ICM'] = ds_hshd['ICM'].replace(0, np.nan)

kcb_cutoffs = pd.read_excel(os.path.join(path1,'kcb_cutoffs.xlsx'))

for col in kcb_cutoffs.columns:
    ds_hshd[col] = kcb_cutoffs.loc[0,col]

income_condition = [
    ds_hshd['ICM'].between(0, ds_hshd['ICM_cutoff1'], inclusive= 'left'),
    ds_hshd['ICM'].between(ds_hshd['ICM_cutoff1'], ds_hshd['ICM_cutoff2'], inclusive= 'left'),
    ds_hshd['ICM'].between(ds_hshd['ICM_cutoff2'], ds_hshd['ICM_cutoff3'], inclusive= 'left'),
    ds_hshd['ICM'].between(ds_hshd['ICM_cutoff3'], ds_hshd['ICM_cutoff4'], inclusive= 'left'),
    ds_hshd['ICM'].between(ds_hshd['ICM_cutoff4'], np.inf, inclusive= 'left')
]

asset_condition = [
    ds_hshd['TOT_ASST'].between(0, ds_hshd['TOT_ASST_cutoff1'], inclusive= 'left'),
    ds_hshd['TOT_ASST'].between(ds_hshd['TOT_ASST_cutoff1'], ds_hshd['TOT_ASST_cutoff2'], inclusive= 'left'),
    ds_hshd['TOT_ASST'].between(ds_hshd['TOT_ASST_cutoff2'], ds_hshd['TOT_ASST_cutoff3'], inclusive= 'left'),
    ds_hshd['TOT_ASST'].between(ds_hshd['TOT_ASST_cutoff3'], ds_hshd['TOT_ASST_cutoff4'], inclusive= 'left'),
    ds_hshd['TOT_ASST'].between(ds_hshd['TOT_ASST_cutoff4'], np.inf, inclusive= 'left')
]

ds_hshd['ICM quantile'] = np.select(income_condition, [1,2,3,4,5], default = np.nan)
ds_hshd['TOT_ASST quantile'] = np.select(asset_condition, [1,2,3,4,5], default = np.nan)
ds_hshd = ds_hshd[['HSHD_SEQNO','ICM quantile','TOT_ASST quantile']]
ds_combined = ds_combined.merge(ds_hshd, on='HSHD_SEQNO', how='left')

print(ds_combined.shape)

In [ ]:
# Individual SNS, Portal Usage

ds_indiv = ds_combined.groupby('KEY').agg({'COMM_SNS_DAY':'mean', 'INFO_PORTAL_DAY':'mean'}).reset_index()
skt_cutoffs = pd.read_excel(os.path.join(path1,'skt_cutoffs.xlsx'))

for col in skt_cutoffs:
    ds_indiv[col] = skt_cutoffs[col].iloc[0]


sns_condition = [
    ds_indiv['COMM_SNS_DAY'].between(ds_indiv['sns_cutoff0'], ds_indiv['sns_cutoff1'], inclusive='left'),
    ds_indiv['COMM_SNS_DAY'].between(ds_indiv['sns_cutoff1'], ds_indiv['sns_cutoff2'], inclusive='left'),
]

portal_condition = [
    ds_indiv['INFO_PORTAL_DAY'].between(ds_indiv['portal_cutoff0'], ds_indiv['portal_cutoff1'], inclusive='left'),
    ds_indiv['INFO_PORTAL_DAY'].between(ds_indiv['portal_cutoff1'], ds_indiv['portal_cutoff2'], inclusive='left'),
]

ds_indiv['sns_quantile'] = np.select(sns_condition, ['1', '2'], default=pd.NA)
ds_indiv['portal_quantile'] = np.select(portal_condition, ['1', '2'], default=pd.NA)


ds_indiv = ds_indiv[['KEY','sns_quantile','portal_quantile']]
ds_combined = ds_combined.merge(ds_indiv, on='KEY', how='left')

In [ ]:
# add 2 digit
ds_combined.drop(columns=['HOME_ADM'], inplace=True)
ds_combined.rename(columns={'HOME_ADM_b':'HOME_ADM'},inplace=True)
ds_combined = uf.add_region(ds_combined, 2, english=True)

In [ ]:
new_var_name ={
    'CD_USE_AMT':'Card Spending AMT',
    'SIN_CD_USE_AMT':'Credit Card Spending AMT',
    'CHK_CD_USE_AMT': 'Debit Card Spending AMT',
    'CD_FUL_USE_AMT': 'Lump-sum Payment AMT',
    'CD_INSTL_USE_AMT':'Installment Payment AMT',
    'CD_CA_USE_AMT':'Cash Advance AMT',
    'CD_ABRD_USE_AMT':'Overseas Card Spending AMT'
}

ds_combined.rename(columns = new_var_name, inplace = True)
ds_combined.to_parquet(os.path.join(path1,'ds_combined.parquet'))